In [1]:
# This is a sample Python script.
import numpy as np
import matplotlib.pyplot as plt

# Press ⌃R to execute it or replace it with your code.
# Press Double ⇧ to search everywhere for classes, files, tool windows, actions, and settings.
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tslearn.metrics import dtw
import string
from sklearn.metrics import balanced_accuracy_score
from database import SpellerDatabase
from database.SpellerDatabase import *
import plot.SpellerPlotter as Plotter
import SpellerConstant
from train import SpellerModel
from sklearn.model_selection import StratifiedKFold
from train import SpellerTrainer
from train import RealTimePeakDetector
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from skl2onnx.common.data_types import FloatTensorType
from sklearn.metrics import balanced_accuracy_score
from skl2onnx import convert_sklearn
import onnxruntime as rt
from skl2onnx.helpers.onnx_helper import save_onnx_model
from sklearn.metrics import confusion_matrix, classification_report
import antropy as ant
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, cross_val_score
from scipy.fftpack import fft, dct



/Users/resitberkaybozkurt/opt/anaconda3/lib/python3.9/site-packages/skl2onnx/algebra/onnx_ops.py:159: UserWarning: OpSchema.FormalParameter.typeStr is deprecated and will be removed in 1.16. Use OpSchema.FormalParameter.type_str instead.
  tys = obj.typeStr or ''
/Users/resitberkaybozkurt/opt/anaconda3/lib/python3.9/site-packages/skl2onnx/algebra/automation.py:154: UserWarning: OpSchema.FormalParameter.isHomogeneous is deprecated and will be removed in 1.16. Use OpSchema.FormalParameter.is_homogeneous instead.
  if getattr(obj, 'isHomogeneous', False):
/Users/resitberkaybozkurt/opt/anaconda3/lib/python3.9/site-packages/jinja2/environment.py:471: UserWarning: OpSchema.FormalParameter.typeStr is deprecated and will be removed in 1.16. Use OpSchema.FormalParameter.type_str instead.
  return getattr(obj, attribute)


In [2]:
def create_label_dict(letter_list: []):
    labels = {}
    for num, letter in enumerate(letter_list, start=1):
        labels[letter] = float(num)
    return labels


In [3]:
TARGET_LETTERS = list(string.ascii_uppercase)
models = ['Summary statistics', 'Sensor data']
balance_acc_score = []
N_SPLITS = 5

In [ ]:
data_frame = SpellerDatabase.read_letters_from_database(SpellerConstant.FIREBASE_REFERENCE,tuple(TARGET_LETTERS))


In [ ]:
def train_session(model,X, y, session_desc):
    # balance acc score list for different datasets
    balance_acc_score = []
    models = []
    train_scores = []
    train_descs = []
    # names of the datasets
    conf_matrices = []
    min = -1
    max = 1

    # train model with scikitlearn

    print("------------ Standardizing data with ({},{}) ({}) ------------".format(min, max, session_desc))
    _X = np.nan_to_num(X, copy=True)
    _X = SpellerPreProcesser.standardize(_X, min, max)
    print("------------ Splitting test,train set ({}) ------------".format(session_desc))
    #leave out test data
    _X, X_valid, y, y_valid = train_test_split(_X, y, test_size=0.2, random_state=42, stratify=y)
    print("------------ Training SciKit Model ({}) ------------".format(session_desc))
    skf = StratifiedKFold(n_splits=N_SPLITS)
    for i, (train_index, test_index) in enumerate(skf.split(_X, y)):
        X_train = _X[train_index]
        y_train = y[train_index]
        X_test = _X[test_index]
        y_test = y[test_index]
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred = np.around(y_pred)
        train_descs.append('Fold {}'.format(i))
        train_scores.append(balanced_accuracy_score(y_test, y_pred))
        
    mean_of_folds = np.mean(train_scores)
    train_scores.append(mean_of_folds)
    train_descs.append('Mean of Folds')
    print("------------ Testing SciKit Model ({}) ------------".format(session_desc))
    y_pred = model.predict(X_valid)
    y_pred = np.around(y_pred)
    train_scores.append(balanced_accuracy_score(y_valid, y_pred))
    train_descs.append('Validation')
    print("------------ Calculating Top-K-Accuray SciKit Model ({}) ------------".format(session_desc))
    #k_values = []
    #score = []
    #for k in TOP_K_VALUES:
     #   k_values.append(k)
      #  score.append(SpellerTrainer.calculate_top_k_accuracy(model, _X, y, k))
    #print("------------ Plotting results of Top-K-Accuracy ------------")
    #plot_bar(score, k_values, "Top K Accuracy Scores Sk {}".format(session_desc))

    conf_matrix_scikit = confusion_matrix(y_valid, y_pred)
    model_desc = "SK-Test-{}".format(session_desc)
    models.append(model_desc)
    conf_matrices.append(conf_matrix_scikit)
    Plotter.plot_conf_matrix(conf_matrix_scikit, 'Prediction', 'Actual', 'Confusion {}'.format(model_desc),
                             TARGET_LETTERS)

    y_pred = model.predict(_X)
    y_pred = np.around(y_pred)
    conf_matrix_scikit = confusion_matrix(y, y_pred)
    model_desc = "SK-Train-{}".format(session_desc)
    models.append(model_desc)
    conf_matrices.append(conf_matrix_scikit)
    Plotter.plot_conf_matrix(conf_matrix_scikit, 'Prediction', 'Actual', 'Confusion {}'.format(model_desc),
                             TARGET_LETTERS)
    
    # Plot performance of two models
    print("------------ Plotting results ------------")
    plot_bar(train_scores, train_descs, "Balance Accuracy Scores")

    return train_descs, train_scores


In [ ]:
def create_summary_statistics(data_frame: pd.DataFrame()):
    for axes in SENSOR_AXES:
        column = data_frame[axes]
        Min = []
        Max = []
        hj_mob_array = []
        hj_com_array = []
        peak_array0 = []
        dip_array0 = []
        peak_array1 = []
        dip_array1 = []
        Mean = []
        Rms = []
        Var = []
        Std = []
        Power = []
        Peak = []
        Skew = []
        Kurtosis = []
        P2p = []
        CrestFactor = []
        FormFactor = []
        PulseIndicator = []
        Max_f = []
        Sum_f = []
        Mean_f = []
        Var_f = []
        Peak_f = []
        Skew_f = []
        Kurtosis_f = []
        data_summ_stat = []
        for X in column:
            ## TIME DOMAIN ##
            Min.append(np.min(X))
            Max.append(np.max(X))
            Mean.append(np.mean(X))
            Rms.append(np.sqrt(np.mean(X ** 2)))
            Var.append(np.var(X))  # Hjorth's activity
            Std.append(np.std(X))
            Power.append(np.mean(X ** 2))
            Peak.append(np.max(np.abs(X)))
            P2p.append(np.ptp(X))
            CrestFactor.append(np.max(np.abs(X)) / np.sqrt(np.mean(X ** 2)))
            Skew.append(stats.skew(X))
            Kurtosis.append(stats.kurtosis(X))
            FormFactor.append(np.sqrt(np.mean(X ** 2)) / np.mean(X))
            PulseIndicator.append(np.max(np.abs(X)) / np.mean(X))
            ## FREQ DOMAIN ##
            ft = fft(X)
            S = np.abs(ft ** 2) / len(X)
            Max_f.append(np.max(S))
            Sum_f.append(np.sum(S))
            Mean_f.append(np.mean(S))
            Var_f.append(np.var(S))
            Peak_f.append(np.max(np.abs(S)))
            Skew_f.append(stats.skew(X))
            Kurtosis_f.append(stats.kurtosis(X))

            ## calculate Hjorth Parameters mobility & complexity ##
            mob, com = ant.hjorth_params(X)
            hj_mob_array.append(mob)
            hj_com_array.append(com)
            ## peak and dips ##
            threshold = (np.max(X) - np.min(X)) / 2
            signal0, signal1 = create_peak_dip_arrays(PEAK_DETECTOR_LAG, threshold, X)
            peak, dip = count_peak_dip(signal0)
            peak_array0.append(peak)
            dip_array0.append(dip)
            peak, dip = count_peak_dip(signal1)
            peak_array1.append(peak)
            dip_array1.append(dip)
            data_summ_stat.append([])

        data_frame['{}_MIN'.format(axes)] = Min
        data_frame['{}_MAX'.format(axes)] = Min
        data_frame['{}_PEAK_INF0'.format(axes)] = peak_array0
        data_frame['{}_DIP_INFf0'.format(axes)] = dip_array0
        data_frame['{}_PEAK_INF1'.format(axes)] = peak_array1
        data_frame['{}_DIP_INF1'.format(axes)] = dip_array1
        data_frame['{}_HJ_MOB'.format(axes)] = hj_mob_array
        data_frame['{}_HJ_COM'.format(axes)] = hj_com_array
        data_frame['{}_MEAN'.format(axes)] = Mean
        data_frame['{}_RMS'.format(axes)] = Rms
        data_frame['{}_VAR'.format(axes)] = Var
        data_frame['{}_STD'.format(axes)] = Std
        data_frame['{}_POWER'.format(axes)] = Power
        data_frame['{}_PEAK'.format(axes)] = Peak
        data_frame['{}_P2P'.format(axes)] = P2p
        data_frame['{}_CREST_FACTOR'.format(axes)] = CrestFactor
        data_frame['{}_SKEW'.format(axes)] = Skew
        data_frame['{}_KURTOSIS'.format(axes)] = Kurtosis
        data_frame['{}_MAX_f'.format(axes)] = Max_f
        data_frame['{}_SUM_f'.format(axes)] = Sum_f
        data_frame['{}_MEAN_f'.format(axes)] = Mean_f
        data_frame['{}_VAR_f'.format(axes)] = Var_f
        data_frame['{}_PEAK_f'.format(axes)] = Peak_f
        data_frame['{}_SKEW_f'.format(axes)] = Skew_f
        data_frame['{}_KURTOSIS_f'.format(axes)] = Kurtosis

        data_frame['data_summ_stat'] = data_summ_stat
    return data_frame


In [ ]:
SUMMARY_STATISTICS_FEATURES = ['MIN', 'MAX', 'PEAK_INF0', 'DIP_INFf0', 'PEAK_INF1', 'DIP_INF1', 'HJ_MOB', 'HJ_COM',
                               'MEAN', 'VAR', 'STD']

In [ ]:
def count_peak_dip(data_array: []):
    peak = sum(1 for i in data_array if i > 0)
    dip = sum(1 for i in data_array if i < 0)
    return peak, dip


def create_peak_dip_arrays(lag, threshold, signal_data):
    realTimePeakDetector0 = RealTimePeakDetector(signal_data[0:lag], PEAK_DETECTOR_LAG, threshold,
                                                 0)
    realTimePeakDetector1 = RealTimePeakDetector(signal_data[0:lag], PEAK_DETECTOR_LAG, threshold,
                                                 1)

    for data_point in signal_data[lag:]:
        realTimePeakDetector0.thresholding_algo(data_point)
        realTimePeakDetector1.thresholding_algo(data_point)

    return realTimePeakDetector0.signals, realTimePeakDetector1.signals

def extract_X_y_summ_sta(data_frame: pd.DataFrame()):
    for axes in SENSOR_AXES:
        for feature in SUMMARY_STATISTICS_FEATURES:
            col_name = '{}_{}'.format(axes, feature)
            # print(type(data_frame[col_name]))
            data_frame['data_summ_stat'] = data_frame['data_summ_stat'].combine(data_frame[col_name],
                                                                                (lambda x1, x2: x1 + [x2]))

    data = data_frame['data_summ_stat'].tolist()
    X = SpellerPreProcesser.standardize(data, -1, 1)
    y = data_frame['target'].to_numpy()
    return X, y


In [ ]:
def plot_bar(y_values, x_values, title):
    x_pos = np.arange(len(x_values))
    y_pos = np.arange(len(y_values))
    plt.bar(y_pos, y_values)
    plt.xticks(x_pos, x_values,rotation=45, ha='right')
    plt.title(title, fontsize=17)
    plt.show()

In [ ]:
from train import SpellerPreProcesser
from SpellerConstant import SENSOR_AXES
from SpellerConstant import PEAK_DETECTOR_LAG
from train.RealTimePeakDetector import RealTimePeakDetector
import scipy.stats as stats
from scipy.fft import fft, fftfreq, dst

# extract X and y values from summ stat data


In [ ]:
data_frame_summ = create_summary_statistics(data_frame)

In [ ]:
X_summ_stat, y_summ_stat = extract_X_y_summ_sta(data_frame_summ)

In [ ]:
data = data_frame_summ[SENSOR_AXES[-2]].iloc[0]
data_dct = dct(data)
data_dst = dst(data)

In [ ]:
plt.plot(data)
plt.show()


In [ ]:
plt.plot(data_dct)
plt.show()


In [ ]:
SUMMARY_STATISTICS_FEATURES


In [ ]:
TOP_K_VALUES = [1, 2, 3, 4, 5]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingRegressor

lr_model = LogisticRegression(random_state=0, multi_class='multinomial')
knn_model = KNeighborsClassifier(n_neighbors=len(TARGET_LETTERS))
hgbr_model = HistGradientBoostingRegressor()

In [ ]:
knn_desc_ss, knn_sc_ss = train_session(knn_model,X_summ_stat, y_summ_stat, "SS_KNN")
lr_desc_ss, lr_sc_ss = train_session(lr_model,X_summ_stat, y_summ_stat, "SS_LR")
hgbr_desc_ss, hgbr_sc_ss = train_session(hgbr_model,X_summ_stat, y_summ_stat, "SS_HGBR")


In [ ]:
sk_model_desc_ss, sk_model_sc_ss

In [ ]:
sklearn_knn.fit(X_train, y_train)


In [ ]:
y_pred = sklearn_knn.predict(X_test)


In [ ]:
balance_acc_score.append(balanced_accuracy_score(y_test, y_pred))

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)


In [ ]:
Plotter.plot_conf_matrix(conf_matrix, 'Prediction', 'Actual', 'Confusion Matrix with SciKit', TARGET_LETTERS)


In [ ]:
from sklearn.metrics import top_k_accuracy_score

In [ ]:
y_prob = sklearn_knn.predict_proba(X_test)

In [ ]:
top_k_accuracy_score(y_test, y_prob, k=5)

In [ ]:
X_train

In [ ]:
y_pred = tslearn_knn.predict(X_test)


In [ ]:
balance_acc_score.append(balanced_accuracy_score(y_test, y_pred))
